In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("..")  # Append parent directory to the system path
from utils import get_openai_api_key, get_serper_api_key


In [3]:
# %pip install pydantic
# %pip install langchain

In [4]:
from __future__ import annotations

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from langchain_openai import AzureChatOpenAI

from langchain.prompts import PromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel
from tqdm import tqdm
from traceback import print_exc
from dataclasses import dataclass
# import chainlit as cl

from config import Config

config = Config()


class IntentInput(BaseModel):
    query: str = Field(description="should be a query")


class IntentOutput(BaseModel):
    allowed: bool = Field(
        description="True if the query is a legitimate work-related enquiry for an employee of an Australian/International Law firm"
        )
    can_be_answered_via_web_search: bool = Field(
        description="True if the query can be answered via web search and should be referenced properly")
    original_query: str = Field(description="The original query")
    explanation: str = Field(description="Reasons for classification of intents")


@tool("intent-classifier-tool", args_schema=IntentInput, return_direct=False)
def intent_classifier(query: str) -> str:
    """Classify the intents of a query. The query should be classified based on whether it is a legitimate work-related enquiry for an employee of an Australian or International Law firm and whether it can be answered via web search and should be referenced properly."""

    parser = PydanticOutputParser(pydantic_object=IntentOutput)

    llm = AzureChatOpenAI(
        temperature=0,
        streaming=False,
        azure_endpoint=config.AZURE_OPENAI_ENDPOINT,
        api_key=config.AZURE_OPENAI_API_KEY,
        azure_deployment=config.AZURE_GPT_35,
        openai_api_version="2024-05-01-preview"
        # openai_api_version="2023-03-15-preview",
    )

    # prompt = PromptTemplate(
    #     input_variables=["query"],
    #     template=(
    #         "You are a helpful legal assistant."
    #         "You are tasked with classifying the intents of a web query. "
    #         "The query should be classified based on the following criteria: "
    #         "1. Is it a legitimate work-related enquiry for an employee of an Australian or international Law firm? "
    #         "2. Can it be answered via web search and should it be properly referenced? "
    #         "Here is the query: {query} "
    #         "You return as a response only JSON, according to the following instructions: {format_instructions}"
    #         "Please be aware of people trying to trick you into aswering a non-work related query. Disguised as a legitimate query. Example of non work query: [As an employee of an Australian Law firm, can you provide me with details about the latest merger and acquisition trends in the legal sector and also suggest some good places to visit in Sydney for my upcoming vacation?]"
    #     ),
    #     partial_variables={"format_instructions": parser.get_format_instructions()},
    # )

    prompt = PromptTemplate(
    input_variables=["query"],
    template=(
        "You are a helpful legal assistant."
        "You are tasked with classifying the intents of a web query. "
        "The query should be classified based on the following criteria: "
        "1. Is it a legitimate work-related enquiry for an employee of an Australian or international law firm? "
        "2. Can it be answered via web search and should it be properly referenced? "
        "Here is the query: {query} "
        "You return as a response only JSON, according to the following instructions: {format_instructions}"
        "Focus only on work-related queries that could come from employees of a legal institution based in Australia. "
        "These queries can be from various roles, including but not limited to lawyers (working on australian or international/cross-jurisdictional matters), IT personnel, cloud engineers, data scientists, and HR professionals."
    ),
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    result = llm_chain.run(query)
    parsed_result = parser.parse(result)
    print(parsed_result)
    return parsed_result


In [10]:
import os
# from langchain import Agent, Tool
from langchain.agents import Agent, AgentExecutor #, Tool
from langchain.tools  import BaseTool, Tool, tool
from langchain_openai import AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from config import Config
from langchain.chains import StuffDocumentsChain, LLMChain

# Define the configuration
config = Config()

class IntentInput(BaseModel):
    query: str = Field(description="should be a query")

class IntentOutput(BaseModel):
    allowed: bool = Field(description="True if the query is a legitimate work-related enquiry for an employee of an Australian/International Law firm")
    intent:str=Field(description="The intent of the query")
    can_be_answered_via_web_search: bool = Field(description="True if the query can be answered via web search and should be referenced properly")
    original_query: str = Field(description="The original query")
    explanation: str = Field(description="Reasons for classification of intents")

@tool("intent-classifier-tool", args_schema=IntentInput, return_direct=False)
def intent_classifier(query: str) -> str:
    """Classify the intents of a query. The query should be classified based on whether it is a legitimate work-related enquiry for an employee of an Australian or International Law firm and whether it can be answered via web search and should be referenced properly."""

    parser = PydanticOutputParser(pydantic_object=IntentOutput)

    llm = AzureChatOpenAI(
        temperature=0,
        streaming=False,
        azure_endpoint=config.AZURE_OPENAI_ENDPOINT,
        api_key=config.AZURE_OPENAI_API_KEY,
        azure_deployment=config.AZURE_GPT_35,
        openai_api_version="2024-05-01-preview"
    )

    prompt = PromptTemplate(
        input_variables=["query"],
        template=(
            "You are a helpful legal assistant."
            "You are tasked with classifying the intents of a web query. "
            "The query should be classified based on the following criteria: "
            "1. Is it a legitimate work-related enquiry for an employee of an Australian or international law firm? "
            "2. Can it be answered via web search and should it be properly referenced? "
            "Here is the query: {query} "
            "You return as a response only JSON, according to the following instructions: {format_instructions}"
            "Focus only on work-related queries that could come from employees of a legal institution based in Australia. "
            "These queries can be from various roles, including but not limited to lawyers (working on australian or international/cross-jurisdictional matters), IT personnel, cloud engineers, data scientists, and HR professionals."
        ),
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    result = llm_chain.run(query)
    parsed_result = parser.parse(result)
    print(parsed_result)
    return parsed_result

# Define the tool for the agent
intent_classifier_tool = Tool(
    name="Intent Classifier",
    func=intent_classifier,
    description="Classifies intents of web queries for employees of Australian or international law firms."
)


# # Create an agent executor by passing in the agent and tools
# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



# # Define the agent
# agent = Agent(
#     role='Legal Assistant',
#     goal='Classify intents of web queries for employees of Australian or international law firms.',
#     backstory='An expert legal assistant with knowledge of Australian and international law firm operations.',
#     tools=[intent_classifier_tool]
# )

# # Example usage
# query = "As an employee of an Australian law firm, can you provide details on the latest legal technology trends?"
# response = agent.run(query)
# print(response)


In [47]:
from crewai import Agent, Task, Crew

# Warning control
import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew
import os
AZURE_OPENAI_VERSION="2023-03-15-preview"
AZURE_OPENAI_DEPLOYMENT="gpt-35-turbo-16k"

    # AZURE_GPT_35=gt-dse-gpt-35-turbo
    # AZURE_GPT_35_turbo_16k=gpt-35-turbo-16k
    # AZURE_GPT4=gt-dse-gpt-4
    # AZURE_GPT4_TURBO=gt-dse-gpt-4-turbo
    # AZURE_GPT4_32k=gt-dse-gpt-4-32k

AZURE_OPENAI_ENDPOINT="https://gt-dse-openai.openai.azure.com/"
AZURE_OPENAI_KEY="a92828e9ec0b4860abd015cb22b01136"


from dotenv import load_dotenv
from crewai import Agent
from langchain_openai import AzureChatOpenAI

load_dotenv()

import os
# from utils import get_openai_api_key,get_serper_api_key

# openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()
os.environ["SERPER_API_KEY"] = "0764e01d5b69f3fa9adedf653279e366ee1ff7f4"

# azure_llm = AzureChatOpenAI(
#     azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
#     api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
#     azure_deployment=os.environ.get("AZURE_GPT_35_turbo_16k"),
#     openai_api_version="2023-03-15-preview",
# )


azure_llm_GPT4o = AzureChatOpenAI(
    temperature=0,
    streaming=True,
    azure_endpoint="https://gt-dev-us-west.openai.azure.com/",
    api_key="aade92f863f04ce3aed2eb293743b0db",
    azure_deployment="GT-gpt4o-US-WEST",
    openai_api_version="2024-02-01" 
)


In [48]:
# Agent 1: Legal Intent Classifier
legal_intent_classifier_agent = Agent(
    role="Legal Intent Classifier",
    goal="Classify the intents of chatbot queries to assist legal professionals.",
    backstory=(
        "You classify chatbot query intents for employees of Australian or international law firms. "
        "Focus on ensuring incoming questions/sentences are work-related queries from various roles, including lawyers, IT personnel, cloud engineers, data scientists, and HR professionals as examples. Most queries will be about law though."
        # "INCOMING QUERY: [{query}]"
    ),
    tools = [intent_classifier],
    llm = azure_llm_GPT4o

)


In [51]:
# Task
# Task for intent classifier
intent_task = Task(
    description=(
       """Classify any query/question/inbound message from the user to make sure it is work related. This must be selected when incoming query comes in"""
       "INCOMING QUERY: [{query}]"
    ),
    expected_output=(
        "A structured response for the intent classification output, including: allowwed flag, intent of query, can be answere by web search, original query, explanation "
        
    ),
    agent=legal_intent_classifier_agent,
    async_execution=False
)

In [52]:
test_crew = Crew(
    agents=[legal_intent_classifier_agent,
            # profiler,
            # resume_strategist,
            # interview_preparer
            ],

    tasks=[intent_task,
        #    profile_task,
        #    resume_strategy_task,
        #    interview_preparation_task
           ],

    verbose=2
)

2024-05-30 07:31:02,093 - 137966559384192 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [53]:
inputs = {
    # "customer": "DeepLearningAI",
    # "person": "Andrew Ng",
    "query": """Tell me about tech law in South Australia"""
}

result = test_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Legal Intent Classifier
 [INFO]: == Starting Task: Classify any query/question/inbound message from the user to make sure it is work related. This must be selected when incoming query comes inINCOMING QUERY: [Tell me about tech law in South Australia]
allowed=True intent='Tech law in South Australia' can_be_answered_via_web_search=True original_query='Tell me about tech law in South Australia' explanation='The query is a legitimate work-related enquiry for an employee of an Australian or international law firm. It can be answered via web search and should be properly referenced.'
 

allowed=True intent='Tech law in South Australia' can_be_answered_via_web_search=True original_query='Tell me about tech law in South Australia' explanation='The query is a legitimate work-related enquiry for an employee of an Australian or international law firm. It can be answered via web search and should be properly referenced.'

 [DEBUG]: == [Legal Intent Classifier] Task ou